In [30]:
pip install -U selenium

In [32]:
pip install beautifulsoup4

# Importing neccessary packages

In [33]:
import requests
import json
import os
import pandas as pd
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.edge.service import Service

# Whether API class

In [34]:
class Weather_API:
    
    def __init__ (self, keyword):
        self.keyword = keyword
        
    def json_print(self, obj):
        # create a formatted string of the Python JSON object
        with open('api_data.txt', 'w') as json_file:
            json.dump(obj, json_file)
        text = json.dumps(obj, sort_keys=True, indent=4)
        print(text)
        
    def create_dataframe(self, obj):
        
        # creating a dataframe from nested JSON objects
        FIELDS = ["source.id", "source.name", "author", "title", "description", "url", "urlToImage", "publishedAt", "content"]
        df = pd.json_normalize(obj['articles'])
        final_df = df[FIELDS]
        #final_df.set_index('source.id', inplace = True)
        display(final_df.head())

    
    def news_api(self):
        
        # Use the news-api to obtain articles published from
        url = ('https://newsapi.org/v2/everything?'
       'q={keyword}&'
       'apiKey=ea28961030d34aa0b129ca7b2fad6389'.format(keyword = self.keyword))
        
        
        response = requests.get(url)
        if (response.status_code == 200):
            print('API call successful!')
            json_response = response.json()
            if(len(json_response['articles']) == 0):
                print('No News Articles Found')
            else:
                
                # Print a String in Json Format
                self.json_print(json_response)
                
                
                
                # Create a pandas DataFrame
                self.create_dataframe(json_response)
                    
        else:
            print('Status code: ', response.status_code)

# Web scrapping class

In [106]:
class Web_Scraping:
    
    def __init__(self, location):
        self.location = location
        
    def selenium_webdriver(self):
        
        serv=Service(r"C:\Users\win7\Downloads\chromedriver_win32\chromedriver.exe")
        # Start the Driver
        driver = webdriver.Chrome(service=serv)
        
        # Hit the url of NASA Earth Data website and wait for 15 seconds.
        url = ('https://earthdata.nasa.gov/search?q={location}'.format(location = self.location))
        driver.get(url)
        time.sleep(15)
        
        # Driver scrolls down 25 times to load the table.
        for i in range(0,30):
            driver.execute_script("window.scrollBy(0,6000)")
            time.sleep(10)
            
        # Fetch the webpage and store in a variable.
        webpage = driver.page_source
        
        # Parse the page using BeautifulSoup
        HTMLPage = BeautifulSoup(webpage, 'html.parser')
        
        titles = []
        description = []
        links = []

        for lists in HTMLPage.find_all(class_ = 'result'):
            if (lists.span.text != '' and len(lists.find_all('p')) != 0):
                titles.append(lists.span.text)
                description.append(lists.find('p', class_ = '').text)
                links.append(lists.find('p', class_ = 'search-link').text)
        
        # Create a DataFrame
        df = pd.DataFrame(list(zip(titles, description, links)),
               columns =['title', 'description', 'link'])
        
       
        
        # Store to csv file
        df.to_csv('webscrap.csv', sep=',', index=False,header=True)
        
        print('Web Scraping Successful!')

## Testing web scrapping using selenium and converting into csv

In [36]:
keyword = input('Enter Keyword to be searched: ').lower()
w_api = Weather_API(keyword)
w_api.news_api()

Enter Keyword to be searched: bangalore
API call successful!
{
    "articles": [
        {
            "author": null,
            "content": "We use cookies and data to<ul><li>Deliver and maintain Google services</li><li>Track outages and protect against spam, fraud, and abuse</li><li>Measure audience engagement and site statistics to unde\u2026 [+1131 chars]",
            "description": "Kohli rages at Bangalore fielding after handing Kolkata a 'freebie'\u00a0\u00a0Reuters",
            "publishedAt": "2023-04-27T05:31:00Z",
            "source": {
                "id": "google-news",
                "name": "Google News"
            },
            "title": "Kohli rages at Bangalore fielding after handing Kolkata a 'freebie' - Reuters",
            "url": "https://consent.google.com/ml?continue=https://news.google.com/rss/articles/CBMib2h0dHBzOi8vd3d3LnJldXRlcnMuY29tL3Nwb3J0cy9jcmlja2V0L2tvaGxpLXJhZ2VzLWJhbmdhbG9yZS1maWVsZGluZy1hZnRlci1oYW5kaW5nLWtvbGthdGEtZnJlZWJpZS0yMDIzLTA0LTI3L9I

,source.id,source.name,author,title,description,url,urlToImage,publishedAt,content
0,google-news,Google News,None,Kohli rages at Bangalore fielding after handin...,Kohli rages at Bangalore fielding after handin...,https://consent.google.com/ml?continue=https:/...,None,2023-04-27T05:31:00Z,We use cookies and data to<ul><li>Deliver and ...
1,bbc-news,BBC News,https://www.facebook.com/bbcnews,Dancing on the grave: The decades-old murder t...,"Shakereh Khaleeli was ""rich and beautiful"". Sh...",https://www.bbc.co.uk/news/world-asia-india-65...,https://ichef.bbci.co.uk/news/1024/branded_new...,2023-05-02T23:07:05Z,"Shakereh Khaleeli was ""rich and beautiful"" and..."
2,None,Boing Boing,Jennifer Sandlin,"Meet the Indian flying fox, the ""big boi of th...",Check out these absolutely gorgeous photos of ...,https://boingboing.net/2023/05/08/meet-the-ind...,https://i0.wp.com/boingboing.net/wp-content/up...,2023-05-08T11:49:26Z,Check out these absolutely gorgeous photos of ...
3,bbc-news,BBC News,None,Listen: IPL - Royal Challengers Bangalore v Ko...,Listen to BBC Radio 5 Sports Extra commentary ...,https://www.bbc.co.uk/sport/live/cricket/65180379,https:////m.files.bbci.co.uk/modules/bbc-morph...,2023-04-25T16:35:02Z,You need one to watch live TV on any channel o...
4,bbc-news,BBC News,None,Listen: IPL - Royal Challengers Bangalore v Ch...,Listen to BBC Radio 5 Sports Extra commentary ...,https://www.bbc.co.uk/sport/live/cricket/65177813,https:////m.files.bbci.co.uk/modules/bbc-morph...,2023-04-16T17:44:27Z,You need one to watch live TV on any channel o...


In [107]:
location = input('Enter Location: ').lower()
ws = Web_Scraping('India')
ws.selenium_webdriver()

Enter Location: india
Web Scraping Successful!
